# Q&A with finetuned BERT

In this section, let's learn how to perform question answering with a finetuned Q&A BERT. First, let us import the necessary modules:

In [23]:
%%capture
!pip install transformers==3.5.1

In [24]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer


Now, we download and load the model. We use the bert-large-uncased-whole-word-masking-finetuned-squad model which is finetuned on the SQUAD (Stanford question answering dataset).


In [25]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Next, we download and load the tokenizer which is used for pretraining the bert-large-uncased-whole-word-masking-finetuned-squad model:


In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


Now that we downloaded the model and tokenizer, let's preprocess the input.

## Preprocessing the input
First, we define the input to the BERT which is question and paragraph text:


In [27]:
question = "What is the immune system?"
paragraph = "The immune system is a system of many biological structures and processes within an organism that protects against disease. To function properly, an immune system must detect a wide variety of agents, known as pathogens, from viruses to parasitic worms, and distinguish them from the organism's own healthy tissue."

Add [CLS] token to the beginning of the question and [SEP] token at the end of both the question and paragraph:

In [28]:
question = '[CLS] ' + question + '[SEP]'
paragraph = paragraph + '[SEP]'


Now, tokenize the question and paragraph:


In [29]:
question_tokens = tokenizer.tokenize(question)
paragraph_tokens = tokenizer.tokenize(paragraph)



Combine the question and paragraph tokens and convert them to input_ids:

In [30]:
tokens = question_tokens + paragraph_tokens
input_ids = tokenizer.convert_tokens_to_ids(tokens)



Next, we define the segment_ids. The segment_ids will be 0 for all the tokens of question and it will be 1 for all the tokens of the paragraph:


In [31]:
segment_ids = [0] * len(question_tokens)
segment_ids += [1] * len(paragraph_tokens)


Now we convert the input_ids and segment_ids to tensor:

In [32]:
input_ids = torch.tensor([input_ids])
segment_ids = torch.tensor([segment_ids])



Now that we processed the input. Let's feed them to the model and get the result.

## Getting the answer
We feed the input_ids and segment_ids to the model which return the start score and end score for all of the tokens:


In [35]:
output = model(input_ids=input_ids, token_type_ids = segment_ids)

In [37]:
start_scores = output.start_logits
end_scores = output.end_logits


Now, we select the start_index which is the index of the token which has a maximum start score and end_index which is the index of the token which has a maximum end score:


In [38]:
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)


That's it! Now, we print the text span between the start and end index as our answer:

In [39]:
print(' '.join(tokens[start_index:end_index+1]))

a system of many biological structures and processes within an organism that protects against disease



Now that we learned how to finetune BERT for the question answering task, in the next section, we learn how to finetune BERT for the named-entity recognition task.

